CoNLL-2003 dataset task demonstrates the labeling of tokens for named entity recognition (NER), part-of-speech (POS) tagging, and chunking. Each component of the JSON object corresponds to a different layer of annotation for the sentence:

1. **Tokens**: These are the individual words or punctuation marks from the text. In this case, the sentence "EU rejects German call to boycott British lamb." is split into tokens:
   - "EU"
   - "rejects"
   - "German"
   - "call"
   - "to"
   - "boycott"
   - "British"
   - "lamb"
   - "."

2. **POS Tags**: This array contains the POS tags corresponding to each token. The tags are encoded as numbers, each representing a specific part of speech (like noun, verb, adjective). These numbers usually correspond to a tagging scheme such as the Penn Treebank POS tags:
   - "EU" is tagged as 22, which represents a proper noun.
   - "rejects" is tagged as 42, indicating a verb in present tense.
   - And so forth.

3. **Chunk Tags**: This array indicates phrase chunk boundaries and types (like NP for noun phrase, VP for verb phrase). Each number again corresponds to a specific type of phrase or boundary in a predefined scheme:
   - "EU" is part of a noun phrase, hence 11.
   - "rejects" begins a verb phrase, indicated by 21.
   - The chunk tags help in parsing the sentence into linguistically meaningful phrases.

4. **NER Tags**: These tags are used for named entity recognition. They identify whether each token is part of a named entity (like a person, location, organization) and the type of entity:
   - "EU" is tagged as 3, denoting an organization.
   - "German" and "British" are tagged as 7, indicating nationality or ethnicity.
   - Other tokens are tagged as 0, meaning they are not recognized as part of any named entity.

In [ ]:
#!pip install uv
#!uv pip install numpy pandas torch transformers datasets scikit-learn umap-learn matplotlib seaborn

Homework:
Load a NER dataset (e.g. CoNLL-2003) using the script provided below.
# Instructions
1. download the **conll2003** from the following [link]("https://data.deepai.org/conll2003.zip")
2. unzip the file
3. download the glove embeddings from [link]("https://huggingface.co/datasets/SLU-CSCI4750/glove.6B.100d.txt/resolve/main/glove.6B.100d.txt.gz")
4. unzip the glove embeddings file
5. update the constants in the code below to point to the correct file paths on your machine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# basic python data science tooling
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from datasets import (
    Dataset,
    DatasetDict,
    Features,
    Sequence,
    ClassLabel,
    Value,
)

# constants for config
LOCAL_DIR = "/content/drive/MyDrive/Data Science/Deep learning for natural language processing/assignments/"
GLOVE_EMBEDS_PATH = '/content/drive/MyDrive/Data Science/Deep learning for natural language processing/assignments/glove.6B.100d.txt'

In [ ]:
train_file = os.path.join(LOCAL_DIR, "train.txt")
valid_file = os.path.join(LOCAL_DIR, "valid.txt")
test_file = os.path.join(LOCAL_DIR, "test.txt")

pos_names = [
    '"',
    "''",
    "#",
    "$",
    "(",
    ")",
    ",",
    ".",
    ":",
    "``",
    "CC",
    "CD",
    "DT",
    "EX",
    "FW",
    "IN",
    "JJ",
    "JJR",
    "JJS",
    "LS",
    "MD",
    "NN",
    "NNP",
    "NNPS",
    "NNS",
    "NN|SYM",
    "PDT",
    "POS",
    "PRP",
    "PRP$",
    "RB",
    "RBR",
    "RBS",
    "RP",
    "SYM",
    "TO",
    "UH",
    "VB",
    "VBD",
    "VBG",
    "VBN",
    "VBP",
    "VBZ",
    "WDT",
    "WP",
    "WP$",
    "WRB",
]

chunk_names = [
    "O",
    "B-ADJP",
    "I-ADJP",
    "B-ADVP",
    "I-ADVP",
    "B-CONJP",
    "I-CONJP",
    "B-INTJ",
    "I-INTJ",
    "B-LST",
    "I-LST",
    "B-NP",
    "I-NP",
    "B-PP",
    "I-PP",
    "B-PRT",
    "I-PRT",
    "B-SBAR",
    "I-SBAR",
    "B-UCP",
    "I-UCP",
    "B-VP",
    "I-VP",
]

ner_names = [
    "O",
    "B-PER",
    "I-PER",
    "B-ORG",
    "I-ORG",
    "B-LOC",
    "I-LOC",
    "B-MISC",
    "I-MISC",
]


def parse_conll(path: str):
    """Parse a CoNLL-2003 file into a list of examples."""
    examples = []
    tokens, pos_tags, chunk_tags, ner_tags = [], [], [], []
    with open(path, encoding="utf-8") as f:
        for line in f:
            if line.startswith("-DOCSTART-") or line.strip() == "":
                if tokens:
                    examples.append(
                        {
                            "tokens": tokens,
                            "pos_tags": pos_tags,
                            "chunk_tags": chunk_tags,
                            "ner_tags": ner_tags,
                        }
                    )
                    tokens, pos_tags, chunk_tags, ner_tags = [], [], [], []
            else:
                splits = line.rstrip().split(" ")
                tokens.append(splits[0])
                pos_tags.append(splits[1])
                chunk_tags.append(splits[2])
                ner_tags.append(splits[3])
    if tokens:
        examples.append(
            {
                "tokens": tokens,
                "pos_tags": pos_tags,
                "chunk_tags": chunk_tags,
                "ner_tags": ner_tags,
            }
        )
    return examples


def as_dataset(examples, features: Features):
    ids = []
    tokens_col, pos_col, chunk_col, ner_col = [], [], [], []
    for i, ex in enumerate(examples):
        ids.append(str(i))
        tokens_col.append(ex["tokens"])
        pos_col.append(ex["pos_tags"])
        chunk_col.append(ex["chunk_tags"])
        ner_col.append(ex["ner_tags"])
    return Dataset.from_dict(
        {
            "id": ids,
            "tokens": tokens_col,
            "pos_tags": pos_col,
            "chunk_tags": chunk_col,
            "ner_tags": ner_col,
        },
        features=features,
    )


features = Features(
    {
        "id": Value("string"),
        "tokens": Sequence(Value("string")),
        "pos_tags": Sequence(ClassLabel(names=pos_names)),
        "chunk_tags": Sequence(ClassLabel(names=chunk_names)),
        "ner_tags": Sequence(ClassLabel(names=ner_names)),
    }
)

train_examples = parse_conll(train_file)
valid_examples = parse_conll(valid_file)
test_examples = parse_conll(test_file)

conll2003 = DatasetDict(
    {
        "train": as_dataset(train_examples, features),
        "validation": as_dataset(valid_examples, features),
        "test": as_dataset(test_examples, features),
    }
)

display(conll2003)

In [ ]:
print(conll2003['train'][2])

In [ ]:
def load_glove_embeddings(file_path, embedding_dim):
    # dict to store word embed vectors
    word_vectors = {}
    with open(file_path, 'r', encoding='utf - 8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = torch.tensor(
                [float(val) for val in values[1:]], dtype=torch.float32)
            word_vectors[word] = vector

    # matrix of embeddings
    vocab_size = len(word_vectors)
    embedding_matrix = torch.zeros((vocab_size, embedding_dim))
    word_to_idx = {}
    idx_to_word = {}
    for i, (word, vector) in enumerate(word_vectors.items()):
        embedding_matrix[i] = vector
        word_to_idx[word] = i
        idx_to_word[i] = word

    return embedding_matrix, word_to_idx, idx_to_word


embedding_dim = 100
embedding_matrix, word_to_idx, idx_to_word = load_glove_embeddings(GLOVE_EMBEDS_PATH, embedding_dim)


embedding_layer = nn.Embedding.from_pretrained(embedding_matrix)
embedding_layer

In [ ]:
UNK_IDX = len(word_to_idx)

def tokens_to_indices(tokens_batch):
    indices = []
    for tokens in tokens_batch:
        idxs = [
            word_to_idx.get(t.lower(), UNK_IDX) for t in tokens
        ]
        indices.append(torch.tensor(idxs, dtype=torch.long))
    return indices


def labels_to_tensors(labels_batch):
    return [torch.tensor(lbls, dtype=torch.long) for lbls in labels_batch]


def collate_fn(batch):
    input_ids = tokens_to_indices([b["tokens"] for b in batch])
    labels = labels_to_tensors([b["ner_tags"] for b in batch])
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=UNK_IDX)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)
    return {"input_ids": input_ids, "labels": labels}


train_dataloader = DataLoader(conll2003["train"], batch_size=32, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(conll2003["validation"], batch_size=32, collate_fn=collate_fn)
test_dataloader = DataLoader(conll2003["test"], batch_size=32, collate_fn=collate_fn)

In [ ]:
print(conll2003["train"][0])

Exercise 1.
  - Create a custom nn.Module class that takes Glove word embeddings as input, passes them through a linear layer, and outputs NER tags
  - Train the model using cross-entropy loss and evaluate its performance using entity-level F1 score
  - Analyze the model's predictions and visualize the confusion matrix to identify common errors

In [ ]:
ner_names = conll2003["train"].features["ner_tags"].feature.names
tag_map   = {i: tag for i, tag in enumerate(ner_names)}
num_tags  = len(ner_names)

class LinearNER(nn.Module):
    def __init__(self, embedding_matrix: torch.Tensor, num_tags: int):

        super().__init__()
        vocab_size, embed_dim = embedding_matrix.shape

        self.embedding = nn.Embedding(vocab_size + 1, embed_dim)
        with torch.no_grad():
            self.embedding.weight[:vocab_size].copy_(embedding_matrix)
            self.embedding.weight[vocab_size].zero_()

        self.classifier = nn.Linear(embed_dim, num_tags)

    def forward(self, input_ids):
        emb = self.embedding(input_ids)
        logits = self.classifier(emb)
        return logits

def compute_metrics(predictions: torch.Tensor, labels: torch.Tensor, tag_map: dict):

    if predictions.ndim > 1:
        predictions = predictions.view(-1)
    if labels.ndim > 1:
        labels = labels.view(-1)

    mask = labels != -100
    if mask.sum().item() == 0:
        return 0.0, 0.0, 0.0

    predictions = predictions[mask]
    labels = labels[mask]

    pred_tags = [tag_map[int(p)] for p in predictions]
    true_tags = [tag_map[int(l)] for l in labels]

    precision = precision_score(true_tags, pred_tags, average='weighted', zero_division=0)
    recall = recall_score(true_tags, pred_tags, average='weighted', zero_division=0)
    f1 = f1_score(true_tags, pred_tags, average='weighted', zero_division=0)

    return precision, recall, f1

def train_linear_model(model, train_dataloader, val_dataloader, tag_map, num_epochs=3):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(num_epochs):

        model.train()
        total_loss = 0.0
        total_preds, total_labels = [], []

        train_bar = tqdm(train_dataloader,
                        desc=f"Epoch {epoch+1}/{num_epochs} Training",
                        unit="batch")
        for b_idx, batch in enumerate(train_bar, start=1):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            total_preds.append(preds.detach().cpu().view(-1))
            total_labels.append(labels.detach().cpu().view(-1))

            running_preds = torch.cat(total_preds, dim=0)
            running_labels = torch.cat(total_labels, dim=0)
            _, _, running_f1 = compute_metrics(running_preds, running_labels, tag_map)
            train_bar.set_postfix(loss=f"{(total_loss/b_idx):.3f}", F1=f"{running_f1:.3f}")

        _, _, epoch_train_f1 = compute_metrics(torch.cat(total_preds), torch.cat(total_labels), tag_map)
        print(f"Epoch {epoch+1} Training - Loss: {total_loss/len(train_dataloader):.4f}, F1: {epoch_train_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []

        val_bar = tqdm(val_dataloader,
                      desc=f"Epoch {epoch+1}/{num_epochs} Validation",
                      unit="batch")
        with torch.no_grad():
            for batch in val_bar:
                input_ids = batch["input_ids"].to(device)
                labels = batch["labels"].to(device)

                logits = model(input_ids)
                loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
                val_loss += loss.item()

                preds = torch.argmax(logits, dim=-1)
                val_preds.append(preds.detach().cpu().view(-1))
                val_labels.append(labels.detach().cpu().view(-1))

        _, _, epoch_val_f1 = compute_metrics(torch.cat(val_preds), torch.cat(val_labels), tag_map)
        print(f"Epoch {epoch+1} Validation - Loss: {val_loss/len(val_dataloader):.4f}, F1 {epoch_val_f1:.4f}\n")

    return model

def evaluate_on_test(model, test_dataloader, tag_map):
    """Valuta il modello sul test set e restituisce predizioni e metriche."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    test_loss = 0.0
    test_preds, test_labels = [], []

    test_bar = tqdm(test_dataloader, desc="Testing", unit="batch")
    with torch.no_grad():
        for batch in test_bar:
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            test_preds.append(preds.cpu().view(-1))
            test_labels.append(labels.cpu().view(-1))

    test_preds_flat = torch.cat(test_preds)
    test_labels_flat = torch.cat(test_labels)

    precision, recall, f1 = compute_metrics(test_preds_flat, test_labels_flat, tag_map)

    print(f"\nTest Results:")
    print(f"  Loss: {test_loss/len(test_dataloader):.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1: {f1:.4f}")

    test_preds_list = [int(x) for x in test_preds_flat.tolist()]
    test_labels_list = [int(x) for x in test_labels_flat.tolist()]

    return test_preds_list, test_labels_list, precision, recall, f1

def analyze_predictions(predictions, labels, tag_map):
    """Analizza e stampa report + errori comuni (ignora padding -100)."""
    labels_np = np.asarray(labels)
    preds_np  = np.asarray(predictions)
    mask = labels_np != -100

    pred_filtered  = preds_np[mask]
    label_filtered = labels_np[mask]

    pred_tags  = [tag_map[p] for p in pred_filtered]
    true_tags  = [tag_map[l] for l in label_filtered]

    print("\n" + "=" * 70)
    print("CLASSIFICATION REPORT:")
    print("=" * 70)
    print(classification_report(true_tags, pred_tags, zero_division=0))

    print("\n" + "=" * 70)
    print("ERRORI PIÙ COMUNI:")
    print("=" * 70)
    errors = {}
    for t, p in zip(true_tags, pred_tags):
        if t != p:
            key = f"{t} -> {p}"
            errors[key] = errors.get(key, 0) + 1
    for error, count in sorted(errors.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"{error:20s}: {count:5d} errori")

    return pred_filtered, label_filtered

def plot_confusion_matrix(predictions, labels, tag_map):

    labels_np = np.asarray(labels)
    preds_np  = np.asarray(predictions)
    mask = labels_np != -100

    pred_filtered  = preds_np[mask]
    label_filtered = labels_np[mask]

    cm = confusion_matrix(label_filtered, pred_filtered)
    tags = list(tag_map.values())

    plt.figure(figsize=(12, 10))
    cm_norm = cm.astype('float') / (cm.sum(axis=1, keepdims=True) + 1e-12)
    sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=tags, yticklabels=tags,
                cbar_kws={'label': 'Proporzione'})
    plt.title('Confusion Matrix - Modello Lineare NER (Normalizzata)', fontsize=14, pad=20)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges',
                xticklabels=tags, yticklabels=tags,
                cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix - Modello Lineare NER (Counts)', fontsize=14, pad=20)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

linear_model = LinearNER(embedding_matrix, num_tags)

print("\nTraining del modello lineare...")
linear_model = train_linear_model(
    linear_model, train_dataloader, val_dataloader, tag_map, num_epochs=3
)

print("\n\nValutazione sul Test Set...")
test_preds, test_labels, linear_precision, linear_recall, linear_f1 = evaluate_on_test(
    linear_model, test_dataloader, tag_map
)

print("\n\nAnalisi delle predizioni sul test set...")
pred_filtered, label_filtered = analyze_predictions(test_preds, test_labels, tag_map)

print("\n\nGenerazione Confusion Matrix...")
plot_confusion_matrix(test_preds, test_labels, tag_map)

Exercise 2. Build a multi-layer perceptron (MLP) for NER using Glove embeddings
   - Extend the previous exercise by creating an nn.Module class that defines an MLP architecture on top of Glove embeddings
   - Experiment with different hidden layer sizes and number of layers
   - Evaluate the trained model using entity-level precision, recall, and F1 scores
   - Compare the performance of the MLP model with the simple linear model from exercise 1

In [ ]:
class MLPNER(nn.Module):
    def __init__(self, embedding_matrix: torch.Tensor, num_tags: int,
                 hidden_sizes=[256, 128], dropout=0.3):

        super(MLPNER, self).__init__()
        vocab_size, embed_dim = embedding_matrix.shape

        self.embedding = nn.Embedding(vocab_size + 1, embed_dim)
        with torch.no_grad():
            self.embedding.weight[:vocab_size].copy_(embedding_matrix)
            self.embedding.weight[vocab_size].zero_()

        layers = []
        input_size = embed_dim

        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_size = hidden_size

        layers.append(nn.Linear(input_size, num_tags))

        self.mlp = nn.Sequential(*layers)

    def forward(self, input_ids):
        emb = self.embedding(input_ids)  # (B, T, D)
        logits = self.mlp(emb)           # (B, T, num_tags)
        return logits


def compute_metrics(predictions, labels, tag_map):

    if predictions.ndim > 1:
        predictions = predictions.view(-1)
    if labels.ndim > 1:
        labels = labels.view(-1)

    mask = labels != -100
    if mask.sum().item() == 0:
        return 0.0, 0.0, 0.0

    predictions = predictions[mask]
    labels = labels[mask]

    pred_tags = [tag_map[int(p)] for p in predictions]
    true_tags = [tag_map[int(l)] for l in labels]

    precision = precision_score(true_tags, pred_tags, average='weighted', zero_division=0)
    recall = recall_score(true_tags, pred_tags, average='weighted', zero_division=0)
    f1 = f1_score(true_tags, pred_tags, average='weighted', zero_division=0)

    return precision, recall, f1

def train_mlp_model(model, train_dataloader, val_dataloader, tag_map, num_epochs=3):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(num_epochs):

        model.train()
        train_loss = 0.0
        train_preds, train_labels = [], []

        train_bar = tqdm(train_dataloader,
                        desc=f"Epoch {epoch+1}/{num_epochs} Training",
                        unit="batch")
        for b_idx, batch in enumerate(train_bar, start=1):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            train_preds.append(preds.detach().cpu().view(-1))
            train_labels.append(labels.detach().cpu().view(-1))

            running_preds = torch.cat(train_preds, dim=0)
            running_labels = torch.cat(train_labels, dim=0)
            _, _, running_f1 = compute_metrics(running_preds, running_labels, tag_map)
            train_bar.set_postfix(loss=f"{(train_loss/b_idx):.3f}", F1=f"{running_f1:.3f}")

        _, _, train_f1 = compute_metrics(torch.cat(train_preds), torch.cat(train_labels), tag_map)
        print(f"Epoch {epoch+1} Training - Loss: {train_loss/len(train_dataloader):.4f}, F1: {train_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []

        val_bar = tqdm(val_dataloader,
                      desc=f"Epoch {epoch+1}/{num_epochs} Validation",
                      unit="batch")
        with torch.no_grad():
            for batch in val_bar:
                input_ids = batch["input_ids"].to(device)
                labels = batch["labels"].to(device)

                logits = model(input_ids)
                loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
                val_loss += loss.item()

                preds = torch.argmax(logits, dim=-1)
                val_preds.append(preds.cpu().view(-1))
                val_labels.append(labels.cpu().view(-1))

        _, _, val_f1 = compute_metrics(torch.cat(val_preds), torch.cat(val_labels), tag_map)
        print(f"Epoch {epoch+1} Validation - Loss: {val_loss/len(val_dataloader):.4f}, F1 {val_f1:.4f}\n")

    return model

def evaluate_on_test(model, test_dataloader, tag_map):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    test_loss = 0.0
    test_preds, test_labels = [], []

    test_bar = tqdm(test_dataloader, desc="Testing", unit="batch")
    with torch.no_grad():
        for batch in test_bar:
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            test_preds.append(preds.cpu().view(-1))
            test_labels.append(labels.cpu().view(-1))

    test_preds_flat = torch.cat(test_preds)
    test_labels_flat = torch.cat(test_labels)

    precision, recall, f1 = compute_metrics(test_preds_flat, test_labels_flat, tag_map)

    print(f"Test Results - Loss: {test_loss/len(test_dataloader):.4f}, "
          f"P: {precision:.4f}, R: {recall:.4f}, F1: {f1:.4f}")

    return precision, recall, f1

configs = [
    {"name": "MLP-Small", "hidden_sizes": [128], "dropout": 0.2},
    {"name": "MLP-Medium", "hidden_sizes": [256, 128], "dropout": 0.3},
    {"name": "MLP-Large", "hidden_sizes": [512, 256, 128], "dropout": 0.3},
    {"name": "MLP-XLarge", "hidden_sizes": [1024, 512, 256, 128], "dropout": 0.3},
]

mlp_results = {}

for config in configs:
    print(f"\n\nAddestramento {config['name']}...")
    print(f"Architettura: {config['hidden_sizes']}, Dropout: {config['dropout']}")

    model = MLPNER(
        embedding_matrix,
        num_tags,
        hidden_sizes=config['hidden_sizes'],
        dropout=config['dropout']
    )

    model = train_mlp_model(model, train_dataloader, val_dataloader, tag_map, num_epochs=3)

    print(f"\nValutazione {config['name']} sul Test Set...")
    precision, recall, f1 = evaluate_on_test(model, test_dataloader, tag_map)

    mlp_results[config['name']] = {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'hidden_sizes': config['hidden_sizes']
    }

mlp_results['Linear'] = {
    'precision': linear_precision,
    'recall': linear_recall,
    'f1': linear_f1,
    'hidden_sizes': None
}

print("\n" + "=" * 70)
print("CONFRONTO PERFORMANCE SUL TEST SET:")
print("=" * 70)
for name, results in mlp_results.items():
    print(f"{name:15s} - F1: {results['f1']:.4f}, "
          f"P: {results['precision']:.4f}, R: {results['recall']:.4f}")

fig, ax = plt.subplots(figsize=(14, 8))

models = list(mlp_results.keys())
f1_scores = [mlp_results[m]['f1'] for m in models]
precision_scores = [mlp_results[m]['precision'] for m in models]
recall_scores = [mlp_results[m]['recall'] for m in models]

x = np.arange(len(models))
width = 0.25

bars1 = ax.bar(x - width, f1_scores, width, label='F1', color='#3498db', alpha=0.8)
bars2 = ax.bar(x, precision_scores, width, label='Precision', color='#2ecc71', alpha=0.8)
bars3 = ax.bar(x + width, recall_scores, width, label='Recall', color='#e74c3c', alpha=0.8)

ax.set_xlabel('Modelli', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Confronto Performance dei Modelli sul Test Set', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(models, rotation=15, ha='right')
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_ylim([0, 1])

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

best_model_name = max(mlp_results.items(), key=lambda x: x[1]['f1'])[0]
best_f1 = mlp_results[best_model_name]['f1']
best_hidden_sizes = mlp_results[best_model_name]['hidden_sizes']

print(f"\n🏆 Modello migliore: {best_model_name} con F1 = {best_f1:.4f}")
if best_hidden_sizes:
    print(f"   Architettura: {best_hidden_sizes}")

Exercise 3. Explore the effects of different activation functions and regularization techniques for NER
  - Modify the MLP model from exercise 2 to allow configurable activation functions (e.g. ReLU, tanh, sigmoid)
  - Train models with different activation functions.
  - Visualize the learned entity embeddings using dimensionality reduction techniques like PCA or t-SNE (edited)

In [ ]:
class ConfigurableMLPNER(nn.Module):
    def __init__(self, embedding_matrix: torch.Tensor, num_tags: int,
                 hidden_sizes=[256, 128], dropout=0.3, activation='relu'):

        super(ConfigurableMLPNER, self).__init__()
        vocab_size, embed_dim = embedding_matrix.shape

        self.embedding = nn.Embedding(vocab_size + 1, embed_dim)
        with torch.no_grad():
            self.embedding.weight[:vocab_size].copy_(embedding_matrix)
            self.embedding.weight[vocab_size].zero_()

        activation_functions = {
            'relu': nn.ReLU,
            'tanh': nn.Tanh,
            'sigmoid': nn.Sigmoid,
            'gelu': nn.GELU,
            'logistic': nn.Sigmoid
        }

        if activation not in activation_functions:
            raise ValueError(f"Activation deve essere una tra: {list(activation_functions.keys())}")

        act_class = activation_functions[activation]

        layers = []
        input_size = embed_dim

        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(act_class())
            layers.append(nn.Dropout(dropout))
            input_size = hidden_size

        layers.append(nn.Linear(input_size, num_tags))
        self.mlp = nn.Sequential(*layers)

    def forward(self, input_ids):
        emb = self.embedding(input_ids)
        logits = self.mlp(emb)
        return logits

def compute_metrics(predictions, labels, tag_map):

    if predictions.ndim > 1:
        predictions = predictions.view(-1)
    if labels.ndim > 1:
        labels = labels.view(-1)

    mask = labels != -100
    if mask.sum().item() == 0:
        return 0.0, 0.0, 0.0

    predictions = predictions[mask]
    labels = labels[mask]

    pred_tags = [tag_map[int(p)] for p in predictions]
    true_tags = [tag_map[int(l)] for l in labels]

    precision = precision_score(true_tags, pred_tags, average='weighted', zero_division=0)
    recall = recall_score(true_tags, pred_tags, average='weighted', zero_division=0)
    f1 = f1_score(true_tags, pred_tags, average='weighted', zero_division=0)

    return precision, recall, f1

def train_model(model, train_dataloader, val_dataloader, tag_map, num_epochs=3):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(num_epochs):

        model.train()
        train_loss = 0.0
        train_preds, train_labels = [], []

        train_bar = tqdm(train_dataloader,
                        desc=f"Epoch {epoch+1}/{num_epochs} Training",
                        unit="batch")
        for b_idx, batch in enumerate(train_bar, start=1):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            train_preds.append(preds.detach().cpu().view(-1))
            train_labels.append(labels.detach().cpu().view(-1))

            running_preds = torch.cat(train_preds, dim=0)
            running_labels = torch.cat(train_labels, dim=0)
            _, _, running_f1 = compute_metrics(running_preds, running_labels, tag_map)
            train_bar.set_postfix(loss=f"{(train_loss/b_idx):.3f}", F1=f"{running_f1:.3f}")

        _, _, train_f1 = compute_metrics(torch.cat(train_preds), torch.cat(train_labels), tag_map)
        print(f"Epoch {epoch+1} Training - Loss: {train_loss/len(train_dataloader):.4f}, F1: {train_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []

        val_bar = tqdm(val_dataloader,
                      desc=f"Epoch {epoch+1}/{num_epochs} Validation",
                      unit="batch")
        with torch.no_grad():
            for batch in val_bar:
                input_ids = batch["input_ids"].to(device)
                labels = batch["labels"].to(device)

                logits = model(input_ids)
                loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
                val_loss += loss.item()

                preds = torch.argmax(logits, dim=-1)
                val_preds.append(preds.cpu().view(-1))
                val_labels.append(labels.cpu().view(-1))

        _, _, val_f1 = compute_metrics(torch.cat(val_preds), torch.cat(val_labels), tag_map)
        print(f"Epoch {epoch+1} Validation - Loss: {val_loss/len(val_dataloader):.4f}, F1 {val_f1:.4f}\n")

    return model

def evaluate_on_test(model, test_dataloader, tag_map):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    test_loss = 0.0
    test_preds, test_labels = [], []

    test_bar = tqdm(test_dataloader, desc="Testing", unit="batch")
    with torch.no_grad():
        for batch in test_bar:
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            test_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            test_preds.append(preds.cpu().view(-1))
            test_labels.append(labels.cpu().view(-1))

    test_preds_flat = torch.cat(test_preds)
    test_labels_flat = torch.cat(test_labels)

    precision, recall, f1 = compute_metrics(test_preds_flat, test_labels_flat, tag_map)

    print(f"Test Results - Loss: {test_loss/len(test_dataloader):.4f}, "
          f"P: {precision:.4f}, R: {recall:.4f}, F1: {f1:.4f}")

    return model, precision, recall, f1

print(f"\nUtilizzo l'architettura del modello migliore dell'esercizio 2: {best_model_name}")
if best_hidden_sizes:
    print(f"Architettura hidden layers: {best_hidden_sizes}")
    use_hidden_sizes = best_hidden_sizes
else:

    use_hidden_sizes = [256, 128]
    print(f"Il modello lineare ha vinto. Uso architettura di default: {use_hidden_sizes}")

activations = ['relu', 'tanh', 'sigmoid', 'gelu', 'logistic']
activation_results = {}

for act in activations:
    print(f"\n\nAddestramento con activation={act}...")

    model = ConfigurableMLPNER(
        embedding_matrix,
        num_tags,
        hidden_sizes=use_hidden_sizes,
        dropout=0.3,
        activation=act
    )

    model = train_model(model, train_dataloader, val_dataloader, tag_map, num_epochs=3)

    print(f"\nValutazione {act} sul Test Set...")
    model, precision, recall, f1 = evaluate_on_test(model, test_dataloader, tag_map)

    activation_results[act] = {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'model': model
    }

print("\n" + "=" * 70)
print("CONFRONTO ACTIVATION FUNCTIONS SUL TEST SET:")
print("=" * 70)
for act, results in activation_results.items():
    print(f"{act.upper():10s} - F1: {results['f1']:.4f}, "
          f"P: {results['precision']:.4f}, R: {results['recall']:.4f}")

fig, ax = plt.subplots(figsize=(14, 8))

activations_list = list(activation_results.keys())
f1_scores = [activation_results[a]['f1'] for a in activations_list]
precision_scores = [activation_results[a]['precision'] for a in activations_list]
recall_scores = [activation_results[a]['recall'] for a in activations_list]

x = np.arange(len(activations_list))
width = 0.25

bars1 = ax.bar(x - width, f1_scores, width, label='F1', color='#9b59b6', alpha=0.8)
bars2 = ax.bar(x, precision_scores, width, label='Precision', color='#f39c12', alpha=0.8)
bars3 = ax.bar(x + width, recall_scores, width, label='Recall', color='#1abc9c', alpha=0.8)

ax.set_xlabel('Activation Functions', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Confronto Activation Functions sul Test Set', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels([a.upper() for a in activations_list], rotation=0)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_ylim([0, 1])

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

best_activation = max(activation_results.items(), key=lambda x: x[1]['f1'])[0]
best_act_f1 = activation_results[best_activation]['f1']
print(f"\n🏆 Migliore activation function: {best_activation.upper()} con F1 = {best_act_f1:.4f}")

def extract_entity_embeddings(model, dataloader, tag_map, device):
    """Estrae gli embeddings per ogni tipo di entità"""
    model.eval()
    entity_embeddings = {tag: [] for tag in tag_map.values() if tag != 'O'}

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Estrazione embeddings"):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            emb = model.embedding(input_ids)

            for i in range(emb.shape[0]):
                for j in range(emb.shape[1]):
                    label_idx = labels[i, j].item()
                    if label_idx != -100:
                        tag = tag_map[label_idx]
                        if tag != 'O':
                            entity_embeddings[tag].append(emb[i, j].cpu().numpy())

    return entity_embeddings


def visualize_embeddings(entity_embeddings, method='pca', n_samples=500):

    all_embeddings = []
    all_labels = []

    for tag, embeddings in entity_embeddings.items():
        if len(embeddings) > 0:
            samples = np.array(embeddings[:n_samples])
            all_embeddings.append(samples)
            all_labels.extend([tag] * len(samples))

    if len(all_embeddings) == 0:
        print("Nessun embedding da visualizzare!")
        return

    all_embeddings = np.vstack(all_embeddings)

    if method == 'pca':
        reducer = PCA(n_components=2)
        reduced = reducer.fit_transform(all_embeddings)
        title = f'PCA - Entity Embeddings (variance: {reducer.explained_variance_ratio_.sum():.2%})'
    else:
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
        reduced = reducer.fit_transform(all_embeddings)
        title = 't-SNE - Entity Embeddings'

    plt.figure(figsize=(12, 8))
    unique_labels = list(set(all_labels))
    colors = plt.cm.tab10(np.linspace(0, 1, len(unique_labels)))

    for label, color in zip(unique_labels, colors):
        mask = np.array(all_labels) == label
        plt.scatter(reduced[mask, 0], reduced[mask, 1],
                   c=[color], label=label, alpha=0.6, s=30)

    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Component 1', fontsize=12)
    plt.ylabel('Component 2', fontsize=12)
    plt.tight_layout()
    plt.show()

print("\n\nVisualizzazione Entity Embeddings...")
best_viz_model = activation_results[best_activation]['model']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

entity_embeddings = extract_entity_embeddings(best_viz_model, val_dataloader, tag_map, device)

print("\nGenerazione plot PCA...")
visualize_embeddings(entity_embeddings, method='pca')

print("\nGenerazione plot t-SNE...")
visualize_embeddings(entity_embeddings, method='tsne')